In [1]:
"""
Plotting a sounding with indices and a hodograph
================================================

"""
def OAX_ReadFile(FILENAME):
    import warnings # Silence the warnings from SHARPpy
    warnings.filterwarnings("ignore")
    import sharppy.plot.skew as skew
    from matplotlib.ticker import ScalarFormatter, MultipleLocator
    from matplotlib.collections import LineCollection
    import matplotlib.transforms as transforms
    import matplotlib.pyplot as plt
    from datetime import datetime
    import numpy as np
    from matplotlib import gridspec
    from sharppy.sharptab import winds, utils, params, thermo, interp, profile
    from sharppy.io.spc_decoder import SPCDecoder

    def decode(filename):

        dec = SPCDecoder(filename)

        if dec is None:
            raise IOError("Could not figure out the format of '%s'!" % filename)

        # Returns the set of profiles from the file that are from the "Profile" class.
        profs = dec.getProfiles()
        stn_id = dec.getStnId()

        for k in list(profs._profs.keys()):
            all_prof = profs._profs[k]
            dates = profs._dates
            for i in range(len(all_prof)):
                prof = all_prof[i]
                new_prof = profile.create_profile(pres=prof.pres, hght=prof.hght, tmpc=prof.tmpc, dwpc=prof.dwpc, wspd=prof.wspd, \
                                                  wdir=prof.wdir, strictQC=False, profile='convective', date=dates[i])
                return new_prof, dates[i], stn_id 

    prof, time, location = decode(FILENAME)
    
    # Plot the parcel trace, but this may fail.  If it does so, inform the user.
#     try:
#         ax.semilogy(prof.mupcl.ttrace, prof.mupcl.ptrace, 'k--')
#     except:
#         print("Couldn't plot parcel traces...")


    srwind = params.bunkers_storm_motion(prof)
    # Calculate indices to be shown.  More indices can be calculated here using the tutorial and reading the params module.
    p1km = interp.pres(prof, interp.to_msl(prof, 1000.))
    p6km = interp.pres(prof, interp.to_msl(prof, 6000.))
    sfc = prof.pres[prof.sfc]
    sfc_1km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p1km)
    sfc_6km_shear = winds.wind_shear(prof, pbot=sfc, ptop=p6km)
    srh3km = winds.helicity(prof, 0, 3000., stu = srwind[0], stv = srwind[1])
    srh1km = winds.helicity(prof, 0, 1000., stu = srwind[0], stv = srwind[1])
    scp = params.scp(prof.mupcl.bplus, prof.right_esrh[0], prof.ebwspd)
    stp_cin = params.stp_cin(prof.mlpcl.bplus, prof.right_esrh[0], prof.ebwspd, prof.mlpcl.lclhght, prof.mlpcl.bminus)
    stp_fixed = params.stp_fixed(prof.sfcpcl.bplus, prof.sfcpcl.lclhght, srh1km[0], utils.comp2vec(prof.sfc_6km_shear[0], prof.sfc_6km_shear[1])[1])
    ship = params.ship(prof)

    # A routine to perform the correct formatting when writing the indices out to the figure.
    def fmt(value, fmt='int'):
#         if fmt == 'int':
#             try:
#                 val = int(value)
#             except:
#                 val = -9999
#         else:
#             try:
#                 val = round(value,3)
#             except:
#                 val = -9999
        try:
            val = round(value,3)
        except:
            val = -9999
        return val

    # Setting a dictionary that is a collection of all of the indices we'll be showing on the figure.
    # the dictionary includes the index name, the actual value, and the units.
    indices = {'SBCAPE': fmt(prof.sfcpcl.bplus),\
               #'SBCIN': [fmt(prof.sfcpcl.bminus), 'J/kg'],\
               'SBLCL': fmt(prof.sfcpcl.lclhght),\
               'SBLFC': fmt(prof.sfcpcl.lfchght),\
               #'SBEL': [fmt(prof.sfcpcl.elhght), 'm AGL'],\
               'SBLI': fmt(prof.sfcpcl.li5),\
               #'MLCAPE': [fmt(prof.mlpcl.bplus), 'J/kg'],\
               #'MLCIN': [fmt(prof.mlpcl.bminus), 'J/kg'],\
               #'MLLCL': [fmt(prof.mlpcl.lclhght), 'm AGL'],\
               #'MLLFC': [fmt(prof.mlpcl.lfchght), 'm AGL'],\
               #'MLEL': [fmt(prof.mlpcl.elhght), 'm AGL'],\
               #'MLLI': [fmt(prof.mlpcl.li5), 'C'],\
               'MUCAPE': fmt(prof.mupcl.bplus),\
               #'MUCIN': [fmt(prof.mupcl.bminus), 'J/kg'],\
               #'MULCL': [fmt(prof.mupcl.lclhght), 'm AGL'],\
               #'MULFC': [fmt(prof.mupcl.lfchght), 'm AGL'],\
               #'MUEL': [fmt(prof.mupcl.elhght), 'm AGL'],\
               'MULI': fmt(prof.mupcl.li5),\
               '0-1 km SRH': fmt(srh1km[0]),\
               '0-1 km Shear': fmt(utils.comp2vec(sfc_1km_shear[0], sfc_1km_shear[1])[1]),\
               '0-3 km SRH': fmt(srh3km[0]),\
               '0-6 km Shear': fmt(utils.comp2vec(sfc_6km_shear[0], sfc_6km_shear[1])[1]),\
               'Eff. SRH': fmt(prof.right_esrh[0]),\
               'EBWD': fmt(prof.ebwspd),\
               #'PWV': [round(prof.pwat, 2), 'inch'],\
               'K-index': fmt(params.k_index(prof)),\
               #'STP(fix)': [fmt(stp_fixed, 'flt'), ''],\
               'SHIP': fmt(ship, 'flt'),\
               'SCP': fmt(scp, 'flt'),\
               'STP(cin)': fmt(stp_cin, 'flt'),\
               'STP(fixed)': fmt(stp_fixed, 'flt'),\
               'FLCAPE': fmt(prof.sfcpcl.bfzl),\
               'BRN': fmt(prof.sfcpcl.brn),\
               'LIMAX': fmt(prof.sfcpcl.limax)\
              }

    # List the indices within the indices dictionary on the side of the plot.
    #trans = transforms.blended_transform_factory(ax.transAxes,ax.transData)

    print('Finished file:',FILENAME)

    
    return indices




In [ ]:
# data_file = 'oax_data/demo1980-01-01 00.oax'
# indices = OAX_ReadFile(data_file)
# indices